## Извлечение диалогов

Ф.М. Достоевский известен психологизмом своих произведений и "взолнованными" диалогами, в которых персонажи многократно повторяют одни и те же реплики и путаются в словах, что создает атмосферу, которую мы и попробуем передать с помощью чат-бота.

In [1]:
import os
import numpy as np
import re

### Сначала подготовим наши данные:

Скачаем тексты произведений Достоевского, которые получится найти

In [2]:
BOOKS = os.listdir('data')
BOOKS

['Достоевский.Бедные_люди.1847.txt',
 'Достоевский.Белые_ночи.1860.txt',
 'Достоевский.Бесы.1873.txt',
 'Достоевский.Вечный_муж.1871.txt',
 'Достоевский.Двойник.1846.txt',
 'Достоевский.Дядюшкин_сон.1860.txt',
 'Достоевский.Игрок_(Из_записок_молодого_человека).1866.txt',
 'Достоевский.Идиот.1874.txt',
 'Достоевский.Неточка_Незванова.1860.txt',
 'Достоевский.Преступление_и_наказание.1867.txt',
 'Достоевский.Село_Степанчиково_и_его_обитатели.1860.txt',
 'Достоевский.Униженные_и_оскорбленные.1861.txt',
 'Достоевский.Хозяйка.1865.txt']

Считаем все тексты книг в одну строку:

In [3]:
data = []
for book in BOOKS:
    with open(f'data/{book}', 'r', encoding='utf-8') as file:
        text = file.read()
    data.append(text)
data = ' '.join(data)

Разобьем текст на строки, убрав пустые. Это поможет нам извлечь диалоги.

In [4]:
textlines = np.array(data.split('\n'))
textlines = textlines[textlines != '']

Фрагмент с диалогом примет следующий вид:

In [5]:
textlines[1005:1015]

array(['   -- Административный восторг? Не знаю что такое.',
       '   -- То-есть... Vous savez chez nous... En un mot, поставьте какую-нибудь самую последнюю ничтожность у продажи каких-нибудь дрянных билетов на железную дорогу, и эта ничтожность тотчас же сочтет себя в праве смотреть на вас Юпитером, когда вы пойдете взять билет, pour vous montrer son pouvoir. "Дай-ка, дескать, я покажу над тобой мою власть"... И это в них до административного восторга доходит... En un mot, я вот прочел, что какой-то дьячок, в одной из наших заграничных церквей, -- mais c\'est tres curieux, -- выгнал, то-есть выгнал буквально из церкви одно замечательное английское семейство, les dames charmantes, пред самым началом великопостного богослужения, -- vous savez ces chants et le livre de Job... единственно под тем предлогом, что "шататься иностранцам по русским церквам есть непорядок, и чтобы приходили в показанное время..." и довел до обморока... Этот дьячок был в припадке административного восторга et

Воспользуемся формальным приближением. Возьмем строки начинающиеся с '-', разобьем их по '--' и будем считать репликой каждый четный элемент.

Так строка: \
    -- Господин,[0] -- сказал он[1], -- вы давно носите красные галстуки?[1] \
И подобные ей будут разбиты корректно. То же касается и отдельных реплик: \
    -- Здравствуйте! [0] \
Реплик с пояснением: \
    -- Почему же?[0] -- спросила она.[1] 

In [6]:
sym = set(['–', '-', '-'])

In [7]:
dialogues = []
position = []
for i, line in enumerate(textlines):
    line = line.strip()
    if len(line) != 0 and line[0] in sym:
        position.append(i)
        dialogues.append(np.array(line.split('--')))

In [8]:
for i in range(len(dialogues)):
    dialogues[i] = dialogues[i][dialogues[i] != '']

In [9]:
res = []
for dialoge in dialogues:
    res.append(''.join(dialoge[::2].tolist()).strip())

Данный подход позволяет получить неплохое приближение быстро и малыми усилиями:

In [10]:
res

['Послушайте,  послушайте, Варвара Алексеевна... знаете ли что, Варвара Алексеевна?..  Видите: вы, как придет день его рождения, возьмите десять книжек и подарите их ему сами, то есть от себя, с свое! стороны; я же возьму тогда одну одиннадцатую и уж тоже подарю от себя, то есть собственно с своей стороны. Так вот, видите ли  Тут старик смешался и замолчал. Я взглянула на него; он с робким ожиданием ожидал моего приговора. "Да зачем же вы хотите, чтоб мы не вместе дарили, Захар Петрович?"  одним словом, старик замешался, покраснел, завяз в своей фразе и не мог сдвинуться с места.',
 'Видите ли,  Я, Варвара Алексеевна, балуюсь подчас... то есть я хочу доложить вам, что я почти и всё балуюсь и всегда балуюсь... придерживаюсь того, что нехорошо... то есть, знаете, этак на дворе такие холода бывают, также иногда неприятности бывают разные, или там как-нибудь грустно сделается, или что-нибудь из нехорошего случится, так я и не удержусь подчас, и забалуюсь, и выпью иногда лишнее. Петруше это

Скопируем функции для подготовки датасета модели GPT HuggingFace.

In [11]:
def get_length_param(text: str, tokenizer) -> str:
    """Maps text to 1 of 4 buckets based on length after encoding.
    Parameters
    ----------
    text: str
        The text to be given 1 of 4 length parameters.

    tokenizer: HuggingFace tokenizer
        Tokenizer that used to compute the length of the text after encoding.
        For more info ee https://huggingface.co/transformers/main_classes/tokenizer.html

    Returns
    -------
    len_param: str
        One of four buckets:
        '1' for short, '2' for medium, '3' for long texts and '-' for all others.
    """
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [12]:
def len_type(tokens_count):
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

Запишем наш датасет в нужном формате в файлы:

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(res, test_size=0.1, random_state=42, shuffle=False)

In [14]:
train_out = []
test_out = []

In [15]:
for i, string in enumerate(train):
    if len_type(len(string.split(' '))) != '-':
        train_out.append(f"|{i%2}|{len_type(len(string.split(' ')))}|{string}</s>")
# for i, string in enumerate(train):
#     train_out.append(f"|{(i+1)%2}|{len_type(len(string.split(' ')))}|{string}</s>")

In [16]:
for i, string in enumerate(test):
    if len_type(len(string.split(' '))) != '-':
        test_out.append(f"|{i%2}|{len_type(len(string.split(' ')))}|{string}</s>")
# for i, string in enumerate(test):
#     test_out.append(f"|{(i+1)}|{len_type(len(string.split(' ')))}|{string}</s>")

In [17]:
def write_data(name, lst):
    with open(f'{name}', 'w', encoding='utf-8') as file:
        for element in lst:
            file.write(element+'\n')

In [18]:
write_data('Dostoevsky_train.txt', train_out)
write_data('Dostoevsky_test.txt', test_out)